In [1]:
!pip install imageio
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 15.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [ ]:
##############################################################################

In [54]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import tensorflow as tf

import os
import numpy as np
import imageio
import matplotlib.pyplot as plt
import pandas as pd
import torch
import sys
import zipfile
from torch.utils import data

from sklearn.model_selection import train_test_split

In [55]:
Dataset = "train"
Dataset1 = "test"
Dataset2 = "flamingo"

with zipfile.ZipFile("../input/tgs-salt-identification-challenge/"+Dataset+".zip","r") as z:
    z.extractall("./")
    
with zipfile.ZipFile("../input/tgs-salt-identification-challenge/"+Dataset1+".zip","r") as z:
    z.extractall("./")
    
with zipfile.ZipFile("../input/tgs-salt-identification-challenge/"+Dataset2+".zip","r") as z:
    z.extractall("./")


In [40]:
class TGSSaltDataset(data.Dataset):
    
    def __init__(self, root_path, file_list):
        self.root_path = root_path
        self.file_list = file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
    
        if index not in range(0, len(self.file_list)):
            return self.__getitem__(np.random.randint(0, self.__len__()))

        file_id = self.file_list[index]
        image_folder = os.path.join(self.root_path, "images")
        image_path = os.path.join(image_folder, file_id + ".png")
        mask_folder = os.path.join(self.root_path, "masks")
        mask_path = os.path.join(mask_folder, file_id + ".png")
        image = np.array(imageio.imread(image_path), dtype=np.uint8)
        mask = np.array(imageio.imread(mask_path), dtype=np.uint8)
        return image, mask

In [41]:
train_mask = pd.read_csv('../input/tgs-salt-identification-challenge/train.csv')

depth = pd.read_csv('../input/tgs-salt-identification-challenge/depths.csv')

train_path = "train/"

file_list = list(train_mask['id'].values)

dataset = TGSSaltDataset(train_path, file_list)

train_images = "../input/tgs-salt-identification-challenge/train/images/"
train_masks = "../input/tgs-salt-identification-challenge/train/masks/"

In [47]:
train_images = "/kaggle/working/train/images/"
train_masks = "/kaggle/working/train/masks/"


In [48]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

train_mask = pd.read_csv('../input/tgs-salt-identification-challenge/train.csv')
depth = pd.read_csv('../input/tgs-salt-identification-challenge/depths.csv')

train_images = "/kaggle/working/train/images/"
train_masks = "/kaggle/working/train/masks/"


ids = next(os.walk(train_images))[2]

# Image size
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 1  # grayscale

X = np.zeros((len(ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y = np.zeros((len(ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool_)

# Load and resize images and masks
for n, id_ in enumerate(ids):
    img_path = os.path.join(train_images, id_)
    mask_path = os.path.join(train_masks, id_)

    img = load_img(img_path, color_mode="grayscale", target_size=(IMG_HEIGHT, IMG_WIDTH))
    img = img_to_array(img)
    img = img / 255.0  # normalize to [0,1]
    X[n] = img

    mask = load_img(mask_path, color_mode="grayscale", target_size=(IMG_HEIGHT, IMG_WIDTH))
    mask = img_to_array(mask)
    mask = mask / 255.0
    Y[n] = mask

print(f"Loaded {len(X)} images.")



Loaded 4000 images.


In [50]:
def build_unet(input_shape):
    inputs = Input(input_shape)
    
    c1 = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    c1 = Conv2D(16, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D()(c1)
    
    c2 = Conv2D(32, 3, activation='relu', padding='same')(p1)
    c2 = Conv2D(32, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D()(c2)
    
    c3 = Conv2D(64, 3, activation='relu', padding='same')(p2)
    c3 = Conv2D(64, 3, activation='relu', padding='same')(c3)
    
    u1 = UpSampling2D()(c3)
    m1 = concatenate([u1, c2])
    c4 = Conv2D(32, 3, activation='relu', padding='same')(m1)
    c4 = Conv2D(32, 3, activation='relu', padding='same')(c4)
    
    u2 = UpSampling2D()(c4)
    m2 = concatenate([u2, c1])
    c5 = Conv2D(16, 3, activation='relu', padding='same')(m2)
    c5 = Conv2D(16, 3, activation='relu', padding='same')(c5)
    
    outputs = Conv2D(1, 1, activation='sigmoid')(c5)
    
    model = Model(inputs, outputs)
    return model


In [51]:
# Split into train and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

model = build_unet((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10, batch_size=32)


2025-04-28 18:01:13.788139: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 16)   │            160 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 128, 128, 16)   │          2,320 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 64, 64, 16)     │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 64, 64, 32)     │          4,640 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 64, 64, 32)     │          9,248 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 32, 32, 32)     │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 32, 32, 64)     │         18,496 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 32, 32, 64)     │         36,928 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d             │ (None, 64, 64, 64)     │              0 │ conv2d_5[0][0]         │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 64, 64, 96)     │              0 │ up_sampling2d[0][0],   │
│                           │                        │                │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 64, 64, 32)     │         27,680 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 64, 64, 32)     │          9,248 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_1           │ (None, 128, 128, 32)   │              0 │ conv2d_7[0][0]         │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 128, 128, 48)   │              0 │ up_sampling2d_1[0][0], │
│ (Concatenate)             │                        │                │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 128, 128, 16)   │          6,928 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)    

 Total params: 117,985 (460.88 KB)

 Trainable params: 117,985 (460.88 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 254s 2s/step - accuracy: 0.7519 - loss: 0.5918 - val_accuracy: 0.7315 - val_loss: 0.5862
Epoch 2/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.7585 - loss: 0.5570 - val_accuracy: 0.7315 - val_loss: 0.5817
Epoch 3/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 247s 2s/step - accuracy: 0.7482 - loss: 0.5663 - val_accuracy: 0.7315 - val_loss: 0.5825
Epoch 4/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.7585 - loss: 0.5540 - val_accuracy: 0.7315 - val_loss: 0.5815
Epoch 5/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.7545 - loss: 0.5577 - val_accuracy: 0.7315 - val_loss: 0.5859
Epoch 6/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.7527 - loss: 0.5599 - val_accuracy: 0.7315 - val_loss: 0.5814
Epoch 7/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.7529 - loss: 0.5597 - val_accuracy: 0.7315 - val_loss: 0.5830
Epoch 8/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.7507 - loss: 0.5635 - val_accu

In [53]:
# Load Test Images
test_ids = next(os.walk(test_images))[2]
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

for n, id_ in enumerate(test_ids):
    img_path = os.path.join(test_images_path, id_)
    img = load_img(img_path, color_mode="grayscale", target_size=(IMG_HEIGHT, IMG_WIDTH))
    img = img_to_array(img)
    img = img / 255.0
    X_test[n] = img

# Predict
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Show a prediction
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(X_test[0].squeeze(), cmap='gray')
plt.title('Input Image')

plt.subplot(1, 2, 2)
plt.imshow(preds_test_t[0].squeeze(), cmap='gray')
plt.title('Predicted Mask')
plt.show()


NameError: name 'test_images_path' is not defined